# 🎬 Sports Video Analysis - Google Colab

Este notebook permite testar seus modelos YOLO e análises de vídeo de futebol no Google Colab.

## 📋 Instruções:
1. Execute as células de cima para baixo (de forma sequencial)
2. Quando pedir para fazer upload, selecione seu vídeo
3. Escolha o modo de análise desejado
4. Baixe o vídeo processado ao final

## 1️⃣ Instalar Dependências

In [ ]:
!pip install opencv-python supervision numpy matplotlib pillow ultralytics tqdm -q
print("✅ Dependências instaladas com sucesso!")

## 2️⃣ Clonar Repositório (ou fazer upload dos arquivos)

In [ ]:
# OPÇÃO A: Clonar do GitHub (descomente a linha abaixo)
# !git clone https://github.com/seu-usuario/sports.git /content/sports-main

# OPÇÃO B: Se você vai fazer upload manual, use o painel de Files no lado esquerdo
# Para isso, clique no ícone de pasta no lado esquerdo e faça upload da pasta 'sports-main'

import os
if os.path.exists('/content/sports-main'):
    print("✅ Repositório encontrado!")
    !ls -la /content/sports-main/
else:
    print("⚠️  Repositório não encontrado. Faça upload da pasta 'sports-main' usando o painel Files no lado esquerdo.")

## 3️⃣ Configurar Python Path e Importar Módulos

In [ ]:
import sys
import os

# Adicionar ao path
sys.path.insert(0, '/content/sports-main')

# Importar módulos do projeto
from sports.configs.soccer import SoccerPitchConfiguration
from sports.annotators.soccer import draw_pitch, draw_points_on_pitch
from sports.common.ball import BallTracker, BallAnnotator
from sports.common.team import TeamClassifier
from sports.common.view import ViewTransformer

# Importar dependências
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO
from tqdm import tqdm
from enum import Enum
from collections import deque
import matplotlib.pyplot as plt
from IPython.display import Video, HTML

print("✅ Todos os módulos importados com sucesso!")

## 4️⃣ Carregar Modelos YOLO

In [ ]:
# Definir caminhos dos modelos
MODEL_DIR = '/content/sports-main/examples/soccer/data'

PLAYER_DETECTION_MODEL_PATH = os.path.join(MODEL_DIR, 'football-player-detection.pt')
PITCH_DETECTION_MODEL_PATH = os.path.join(MODEL_DIR, 'football-pitch-detection.pt')
BALL_DETECTION_MODEL_PATH = os.path.join(MODEL_DIR, 'football-ball-detection.pt')

# Verificar se os modelos existem
print("\n📁 Verificando modelos:")
print(f"Player detection: {'✅' if os.path.exists(PLAYER_DETECTION_MODEL_PATH) else '❌'} {PLAYER_DETECTION_MODEL_PATH}")
print(f"Pitch detection: {'✅' if os.path.exists(PITCH_DETECTION_MODEL_PATH) else '❌'} {PITCH_DETECTION_MODEL_PATH}")
print(f"Ball detection: {'✅' if os.path.exists(BALL_DETECTION_MODEL_PATH) else '❌'} {BALL_DETECTION_MODEL_PATH}")

# Carregar modelos
print("\n⏳ Carregando modelos... (pode levar alguns minutos)")
player_detector = YOLO(PLAYER_DETECTION_MODEL_PATH)
pitch_detector = YOLO(PITCH_DETECTION_MODEL_PATH)
ball_detector = YOLO(BALL_DETECTION_MODEL_PATH)

print("✅ Modelos carregados com sucesso!")

## 5️⃣ Fazer Upload do Vídeo

In [ ]:
from google.colab import files

print("\n📹 Selecione seu arquivo de vídeo...")
print("Formatos suportados: .mp4, .avi, .mov, .mkv")

uploaded = files.upload()
video_path = list(uploaded.keys())[0]

print(f"\n✅ Vídeo enviado: {video_path}")
print(f"Tamanho: {os.path.getsize(video_path) / (1024**2):.2f} MB")

# Verificar info do vídeo
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

print(f"\n📊 Informações do vídeo:")
print(f"  Resolução: {width}x{height}")
print(f"  FPS: {fps}")
print(f"  Frames: {frame_count}")
print(f"  Duração: {frame_count/fps:.2f}s")

## 6️⃣ Selecionar Modo de Análise

In [ ]:
# Definir modo de análise
class Mode(Enum):
    PLAYER_DETECTION = "player_detection"
    BALL_DETECTION = "ball_detection"
    PITCH_DETECTION = "pitch_detection"
    PLAYER_TRACKING = "player_tracking"
    COMBINED_ANALYSIS = "combined_analysis"

# Escolha o modo (mude aqui para testar diferentes modos)
selected_mode = Mode.PLAYER_DETECTION  # Altere para: BALL_DETECTION, PITCH_DETECTION, COMBINED_ANALYSIS, etc.

print(f"\n🎯 Modo selecionado: {selected_mode.value.upper()}")
print("\n📝 Modos disponíveis:")
for mode in Mode:
    print(f"  - {mode.value}")
print(f"\n💡 Para mudar de modo, edite a linha 'selected_mode = Mode.PLAYER_DETECTION' acima")

## 7️⃣ Processar Vídeo

In [ ]:
# Configurações
STRIDE = 30  # Processar a cada N frames (aumentar para mais rápido, diminuir para mais preciso)
CONFIDENCE = 0.5  # Confiança mínima para detecções

output_path = '/content/output_video.mp4'
os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)

# Abrir vídeo de entrada
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Criar writer para vídeo de saída
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print(f"\n⏳ Processando vídeo...")
print(f"  Modo: {selected_mode.value}")
print(f"  Stride: {STRIDE} frames")
print(f"  Total de frames: {frame_count}")

frame_id = 0
pbar = tqdm(total=frame_count, desc="Processando")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Processar a cada N frames
    if frame_id % STRIDE == 0:
        if selected_mode == Mode.PLAYER_DETECTION:
            # Detectar jogadores
            results = player_detector.predict(frame, conf=CONFIDENCE, verbose=False)
            detections = sv.Detections.from_ultralytics(results[0])
            frame = sv.BoxAnnotator().annotate(scene=frame, detections=detections)
            frame = sv.LabelAnnotator().annotate(scene=frame, detections=detections)
        
        elif selected_mode == Mode.BALL_DETECTION:
            # Detectar bola
            results = ball_detector.predict(frame, conf=CONFIDENCE, verbose=False)
            detections = sv.Detections.from_ultralytics(results[0])
            frame = sv.CircleAnnotator(radius=10).annotate(scene=frame, detections=detections)
        
        elif selected_mode == Mode.PITCH_DETECTION:
            # Detectar campo
            results = pitch_detector.predict(frame, conf=CONFIDENCE, verbose=False)
            detections = sv.Detections.from_ultralytics(results[0])
            frame = sv.BoxAnnotator().annotate(scene=frame, detections=detections)
        
        elif selected_mode == Mode.COMBINED_ANALYSIS:
            # Análise combinada: Jogadores + Bola
            player_results = player_detector.predict(frame, conf=CONFIDENCE, verbose=False)
            ball_results = ball_detector.predict(frame, conf=CONFIDENCE, verbose=False)
            
            player_detections = sv.Detections.from_ultralytics(player_results[0])
            ball_detections = sv.Detections.from_ultralytics(ball_results[0])
            
            # Anotar jogadores
            frame = sv.BoxAnnotator(color=sv.Color.BLUE).annotate(
                scene=frame, detections=player_detections
            )
            # Anotar bola
            frame = sv.CircleAnnotator(radius=8, color=sv.Color.RED).annotate(
                scene=frame, detections=ball_detections
            )
    
    # Escrever frame no vídeo de saída
    out.write(frame)
    frame_id += 1
    pbar.update(1)

cap.release()
out.release()
pbar.close()

print(f"\n✅ Vídeo processado com sucesso!")
print(f"📁 Salvo em: {output_path}")
print(f"📊 Tamanho: {os.path.getsize(output_path) / (1024**2):.2f} MB")

## 8️⃣ Visualizar Resultado

In [ ]:
# Exibir vídeo processado
print("\n▶️  Exibindo vídeo processado...\n")
Video(output_path, width=800)

## 9️⃣ Baixar Vídeo Processado

In [ ]:
print("\n📥 Preparando para download...\n")
files.download(output_path)
print(f"\n✅ Download iniciado!")
print(f"📁 Arquivo: output_video.mp4")

## 🔧 Opções Avançadas

### Ajustar Parâmetros e Reprocessar

In [ ]:
# Você pode copiar a célula 7 (Processar Vídeo) aqui e ajustar os parâmetros:
# - STRIDE: aumentar para processar menos frames (mais rápido)
# - CONFIDENCE: aumentar para menos detecções falsas
# - selected_mode: trocar o modo de análise

print("\n💡 Dicas para ajustar:")
print("\n1. Aumentar STRIDE (ex: 60) para processar mais rápido")
print("2. Aumentar CONFIDENCE (ex: 0.7) para reduzir falsos positivos")
print("3. Mudar selected_mode para testar diferentes modos")
print("4. Copie a célula 7 inteira, ajuste os parâmetros e execute novamente")

### Ver Primeiros Frames do Resultado

In [ ]:
# Extrair e exibir primeiros frames
cap = cv2.VideoCapture(output_path)
frames_to_show = 3

fig, axes = plt.subplots(1, frames_to_show, figsize=(15, 5))

for i in range(frames_to_show):
    ret, frame = cap.read()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        axes[i].imshow(frame_rgb)
        axes[i].set_title(f"Frame {i*STRIDE}")
        axes[i].axis('off')

cap.release()
plt.tight_layout()
plt.show()

print(f"\n✅ Primeiros frames do resultado exibidos!")

## 📚 Guia de Uso Rápido

### Primeiro uso:
1. Execute **célula 1** → Instala dependências
2. Execute **célula 2** → Faz upload ou clona repositório
3. Execute **célula 3** → Carrega módulos
4. Execute **célula 4** → Carrega modelos YOLO
5. Execute **célula 5** → Faz upload do vídeo
6. Execute **célula 6** → Seleciona modo (edite se quiser mudar)
7. Execute **célula 7** → Processa vídeo (⏳ pode demorar)
8. Execute **célula 8** → Visualiza resultado
9. Execute **célula 9** → Baixa vídeo processado

### Próximos usos:
- Pule as células 1-4 (já estão instaladas/carregadas)
- Faça upload de novo vídeo na célula 5
- Ajuste modo/parâmetros e reprocesse

### 💡 Dicas de Velocidade:
- Ative **GPU** no Colab: Runtime → Change runtime type → GPU
- Aumente `STRIDE` (ex: 60) para processar menos frames
- Use vídeos menores para testes rápidos
- Aumente `CONFIDENCE` para menos detecções